- Training vaccine twitter data: broadly testing different classifiers and parameters without final prediction.
- by Xiaoyi Yuan,
- July 2017

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
# df=pd.read_json('/content/gdrive/Shareddrives/Data Mining Project/english_tweets/en_2021-02-12_clean-dataset.jsonl', orient = 'records', lines = True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
!ls './gdrive/MyDrive/Sem 3/DM/vacc/labeled_data.csv'

'./gdrive/MyDrive/Sem 3/DM/vacc/labeled_data.csv'


In [12]:
import numpy as np
import pandas as pd
pd.options.display.max_rows=100
pd.options.display.max_colwidth=200
from matplotlib import pyplot as plt
%matplotlib inline
import datetime
import os

In [13]:
df = pd.read_csv('./gdrive/MyDrive/Sem 3/DM/vacc/labeled_data.csv')

In [14]:
df

,text,label
0,Vaccine Skeptics Find Unexpected Allies in Conservative GOP http://t.co/HMprooeVg5,1
1,RT @CiiaqapLFraser: If we know there 5% chance of vaccine failure we should know by now whom is vulnerable to vaccine injury. #vaccineinjur…,-1
2,"Homeopathic vaccine warning labels not enough, doctors tell Health Canada https://t.co/SDBPnSZbjE",1
3,"“@ForbesTech: The anti-vaccine movement has caused the worst measles outbreak in 20 years: http://t.co/zaRMf1kPkB” ""That is misinformation!""",-1
4,RT @guardianscience: Measles vaccination rates in parts of Africa surpass those in North America http://t.co/yrTXZohwTW,1
...,...,...
1400,#listen #learnit #jimmykimmel A Message for the Anti-Vaccine Movement: http://t.co/u8v2P0Yg1X via @YouTube,1
1401,#Italian Ministry of Health pays out compensation on a #Vaccine that gave a child #Autism and brain damage http://t.co/mK2GLY1iny,-1
1402,I liked a @YouTube video http://t.co/KHK8snlf1k Rand Paul Ignites Vaccine Controversy,0
1403,#health Flu vaccine is ‘barely effective’: This year's seasonal flu vaccine is barely able to prevent the main... http://t.co/M3qHerdidG,0


# Import and create dataframe

In [ ]:
# path ="/Users/Charlotte/Google Drive/Jaxy Project/training"
# data_xy=pd.read_csv(os.path.join(path,"sample_xy_07_28.csv"))
# data_ja=pd.read_csv(os.path.join(path,"sample_ja_07_28.csv"))
# n_labeled_xy=807
# n_labeled_ja=630

In [ ]:
# #double check the number for each label. 

# dfv_xy=data_xy['label'].value_counts(dropna=False)
# dfv_ja=data_ja['label'].value_counts(dropna=False)

# #check how many data is being read but not labeled (skipped data)
# null_xy=data_xy[:n_labeled_xy]["label"].isnull().value_counts()
# null_ja=data_ja[:n_labeled_ja]["label"].isnull().value_counts()

# print(dfv_xy)
# print(dfv_ja)
# print(null_xy)
# print(null_ja)

In [ ]:
# #get the labeled data
# data_xy=data_xy[pd.notnull(data_xy["label"])]
# data_ja=data_ja[pd.notnull(data_ja["label"])]

# #Put xy and ja labeled data together
# data=data_xy.append(data_ja)

In [ ]:
# #get the data with labels of either 1 or -1
# data_xy_binary=data_xy[data_xy.label!= 0]
# data_ja_binary=data_ja[data_ja.label!= 0]


In [22]:
# print(len(data_xy_binary))
# print(len(data_ja_binary))
data = df

In [17]:
data_binary=df[df.label!= 0]

In [18]:
data_binary

,text,label
0,Vaccine Skeptics Find Unexpected Allies in Conservative GOP http://t.co/HMprooeVg5,1
1,RT @CiiaqapLFraser: If we know there 5% chance of vaccine failure we should know by now whom is vulnerable to vaccine injury. #vaccineinjur…,-1
2,"Homeopathic vaccine warning labels not enough, doctors tell Health Canada https://t.co/SDBPnSZbjE",1
3,"“@ForbesTech: The anti-vaccine movement has caused the worst measles outbreak in 20 years: http://t.co/zaRMf1kPkB” ""That is misinformation!""",-1
4,RT @guardianscience: Measles vaccination rates in parts of Africa surpass those in North America http://t.co/yrTXZohwTW,1
...,...,...
1397,RT @HealthRanger: Pro-#vaccine fanatics grasping at straws: dead children the latest ploy in push for #MandatoryVaccinations - http://t.co/…,-1
1398,Rona Ambrose says anti-vaccine movement puts children at risk - CBC.ca http://t.co/LUP56h1rky,1
1399,Walking in the health service like I didn't care about the vaccination but my heart was beating insanely fast 💉😳,1
1400,#listen #learnit #jimmykimmel A Message for the Anti-Vaccine Movement: http://t.co/u8v2P0Yg1X via @YouTube,1


# Clean texts

In [19]:
#remove url,non-sensical words, emojis and @ (kept the content of hashtags).

import re
import string
import nltk

def clean_text(text):
#remove the emoji or other weird content (such as ðŸ‡ºðŸ‡) 
    text = ''.join(filter(lambda x: x in string.printable, text))
#remove urls and @
    text = re.sub(r"http\S+", "", text)
    text = re.sub("RT","",text)
    text = ' '.join(filter(lambda x:x[0]!="@",text.split()))
    text = ' '.join(filter(lambda x:x[0]!="&",text.split()))
    text= " ".join(list(map(lambda x:x.strip("#"),text.split()))) 
    return text

In [23]:
# replace the text with its cleaned version
data["text"]=data["text"].map(clean_text)
data_binary["text"]=data_binary["text"].map(clean_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# export a copy using today's date as the file name
'''
date=datetime.date.today()
file_name = os.path.join(path, str(date) + "." + "csv")
data.to_csv(file_name)
'''

'\ndate=datetime.date.today()\nfile_name = os.path.join(path, str(date) + "." + "csv")\ndata.to_csv(file_name)\n'

In [42]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 4.7 MB/s 
     |████████████████████████████████| 3.1 MB 16.8 MB/s 
     |████████████████████████████████| 3.3 MB 50.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.7 MB/s 
     |████████████████████████████████| 61 kB 538 kB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=10f9eb466613b8c4d619d9b8e74b872b1a2d4011e926c45e289637207c0563a6
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [43]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [112]:
import torch

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()

        self.linear1 = torch.nn.Linear(input_dim, input_dim)
        self.m = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(input_dim, input_dim//2)
        self.linear3 = torch.nn.Linear(input_dim//2, output_dim)
        
    def forward(self, x):
        # x = sbert_model.encode([x])[0]
        x = self.m(self.linear2(self.m(self.linear1(x))))
        outputs = torch.sigmoid(self.linear3(x))
        return outputs

In [46]:
data_binary['text'] = data_binary.apply(lambda row: sbert_model.encode(row['text']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
X_test = X_test.reset_index(drop=True)

In [70]:
# import numpy as np 
# from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
# sss = ShuffleSplit(n_splits=1, test_size=0.2)

# data_size = len(data_binary)
# X = data_binary["text"]
# y = data_binary["label"]

# sss.get_n_splits(X, y)
# train_index, test_index = next(sss.split(X, y)) 

# X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
# y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
# X_test[0]

In [91]:
X_train, X_test = torch.from_numpy(np.stack(X_train)),torch.from_numpy(np.stack(X_test))

In [110]:
data_binary = data_binary.replace({'label':{-1:0}})

In [114]:
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib
from tqdm import tqdm
import torch

# matplotlib.rc('text', usetex=True)
# matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
# sns.set_style("darkgrid")

# from sklearn.datasets import make_classification
# import matplotlib.pyplot as plt

# separable = False
# while not separable:
#     samples = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_informative=1, n_clusters_per_class=1, flip_y=-1)
#     red = samples[0][samples[1] == 0]
#     blue = samples[0][samples[1] == 1]
#     separable = any([red[:, k].max() < blue[:, k].min() or red[:, k].min() > blue[:, k].max() for k in range(2)])


# red_labels = np.zeros(len(red))
# blue_labels = np.ones(len(blue))

# labels = np.append(red_labels,blue_labels)
# inputs = np.concatenate((red,blue),axis=0)
# import numpy as np 
# from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
# sss = ShuffleSplit(n_splits=1, test_size=0.2)

# data_size = len(data_binary)
# X = data_binary["text"]
# y = data_binary["label"]

# sss.get_n_splits(X, y)
# train_index, test_index = next(sss.split(X, y)) 

# X_train, X_test = X[train_index], X[test_index] 
# y_train, y_test = y[train_index], y[test_index]
X_train,X_test, y_train,y_test=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
# X_train, X_test, y_train,  y_test = train_test_split(
#     inputs, labels, test_size=0.33, random_state=42)

# class LogisticRegression(torch.nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(LogisticRegression, self).__init__()
#         self.linear = torch.nn.Linear(input_dim, output_dim)
        
#     def forward(self, x):
#         outputs = torch.sigmoid(self.linear(x))
#         return outputs

epochs = 1000
input_dim = 768 # Two inputs x1 and x2 
output_dim = 1 # Two possible outputs
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim)

criterion = torch.nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

X_train, X_test = torch.from_numpy(np.stack(X_train)).type(torch.float),torch.from_numpy(np.stack(X_test)).type(torch.float)
y_train, y_test = torch.from_numpy(np.stack(y_train)).type(torch.float),torch.from_numpy(np.stack(y_test)).type(torch.float)

losses = []
losses_test = []
Iterations = []
iter = 0
for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
    x = X_train
    labels = y_train
    optimizer.zero_grad() # Setting our stored gradients equal to zero
    outputs = model(X_train)
    loss = criterion(torch.squeeze(outputs), labels) # [200,1] -squeeze-> [200]
    
    loss.backward() # Computes the gradient of the given tensor w.r.t. graph leaves 
    
    optimizer.step() # Updates weights and biases with the optimizer (SGD)
    
    iter+=1
    if iter%50==0:
        # calculate Accuracy
        with torch.no_grad():
            # Calculating the loss and accuracy for the test dataset
            correct_test = 0
            total_test = 0
            outputs_test = torch.squeeze(model(X_test))
            loss_test = criterion(outputs_test, y_test)
            
            predicted_test = outputs_test.round().detach().numpy()
            total_test += y_test.size(0)
            correct_test += np.sum(predicted_test == y_test.detach().numpy())
            accuracy_test = 100 * correct_test/total_test
            losses_test.append(loss_test.item())
            
            # Calculating the loss and accuracy for the train dataset
            total = 0
            correct = 0
            total += y_train.size(0)
            correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == y_train.detach().numpy())
            accuracy = 100 * correct/total
            losses.append(loss.item())
            Iterations.append(iter)
            
            print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")


# def model_plot(model,X,y,title):
#     parm = {}
#     b = []
#     for name, param in model.named_parameters():
#         parm[name]=param.detach().numpy()  
    
#     w = parm['linear.weight'][0]
#     b = parm['linear.bias'][0]
#     plt.scatter(X[:, 0], X[:, 1], c=y,cmap='jet')
#     u = np.linspace(X[:, 0].min(), X[:, 0].max(), 2)
#     plt.plot(u, (0.5-b-w[0]*u)/w[1])
#     plt.xlim(X[:, 0].min()-0.5, X[:, 0].max()+0.5)
#     plt.ylim(X[:, 1].min()-0.5, X[:, 1].max()+0.5)
#     plt.xlabel(r'$\boldsymbol{x_1}$',fontsize=16) # Normally you can just add the argument fontweight='bold' but it does not work with latex
#     plt.ylabel(r'$\boldsymbol{x_2}$',fontsize=16)
#     plt.title(title)
#     plt.show()


# # Train Data
# model_plot(model,X_train,y_train,'Train Data')

# # Test Dataset Results
# model_plot(model,X_test,y_test,'Test Data')

Training Epochs:   5%|▌         | 52/1000 [00:03<00:55, 17.05it/s]

Iteration: 50. 
Test - Loss: 0.6772093772888184. Accuracy: 58.490566037735846
Train -  Loss: 0.6694067716598511. Accuracy: 61.06508875739645



Training Epochs:  10%|█         | 102/1000 [00:05<00:53, 16.94it/s]

Iteration: 100. 
Test - Loss: 0.6706644892692566. Accuracy: 58.490566037735846
Train -  Loss: 0.6573483347892761. Accuracy: 61.06508875739645



Training Epochs:  15%|█▌        | 152/1000 [00:08<00:49, 17.18it/s]

...

Iteration: 900. 
Test - Loss: 0.4915623962879181. Accuracy: 78.30188679245283
Train -  Loss: 0.4339044988155365. Accuracy: 81.77514792899409



Training Epochs:  95%|█████████▌| 952/1000 [00:54<00:02, 17.13it/s]

Iteration: 950. 
Test - Loss: 0.48217374086380005. Accuracy: 79.24528301886792
Train -  Loss: 0.42180135846138. Accuracy: 82.60355029585799



Training Epochs: 100%|██████████| 1000/1000 [00:57<00:00, 17.31it/s]

Iteration: 1000. 
Test - Loss: 0.47368982434272766. Accuracy: 80.66037735849056
Train -  Loss: 0.40998002886772156. Accuracy: 83.4319526627219



In [122]:
torch.save(model, 'model.pickle')
model = torch.load('model.pickle')
model.eval()

LogisticRegression(
  (linear1): Linear(in_features=768, out_features=768, bias=True)
  (m): ReLU()
  (linear2): Linear(in_features=768, out_features=384, bias=True)
  (linear3): Linear(in_features=384, out_features=1, bias=True)
)

In [121]:
df=pd.read_json('/content/gdrive/Shareddrives/Data Mining Project/english_tweets/en_2021-02-12_clean-dataset.jsonl', orient = 'records', lines = True)

In [123]:
df = df[df['full_text'].str.contains('vaccin') | df['full_text'].str.contains('vax')]
df.drop(df.columns.difference(['id','full_text', 'user']), 1, inplace=True)
user_data = pd.json_normalize(df['user'])
user_data=user_data.rename(columns={"id":"user_id"})
df = df.join(user_data)
df.drop(df.columns.difference(['id','full_text', 'location']), 1, inplace=True)
df["full_text"]=df["full_text"].map(clean_text)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,full_text,location
8,1360091975953895400,Can Canada get some vaccines. US is administering a million vaccines per day. Wtf are we doing COVID19 Canada,"Detroit, Michigan, USA"
11,1360092153062580200,"Kenyans might have to wait for about a year to get the much-awaited Covid-19 vaccine. According to a draft vaccination rollout plan, the first group will get a shot in the arm in January 2022.","New Britain, CT"
14,1360092209576431600,Senate Majority Leader Migz Zubiri on Friday defended his bill exempting local government units from procurement rules and allowing them to directly purchase Covid-19 vaccines from the manufacture...,"San Francisco, CA"
19,1360092078911422500,Coronavirus live updates: Fauci predicts vaccine open season by April,San Diego & Chula Vista
20,1360091994815557600,"What we think as COVID19 vaccine side effects are actually a good thing, according to this video","Omaha, NE"
...,...,...,...
147420,1360454503502286800,"Novant Health, Hornets team up to give 2,200 Covid-19 vaccine doses at Spectrum Center - Charlotte Business Journal",NaN
147424,1360454647283032000,"BREAKING Frederick K.C. Price, founder of the Crenshaw Christian Center has lost his battle with COVID19. In social media posts, Price's family has asked for time to process their loss. The CCC wa...",NaN
147427,1360454419062612000,President Joe Biden is working to make sure this Countrys people are free of the Coronavirus and getting vaccinations out to the people and getting the economy back ! He care about all people! we ...,NaN
147430,1360454522208997400,"This is a slide from the ACIP COVID-19 Vaccines Work Group (12/20/20 Dooling) There are more than 110 million people, aged 16 64 years old, who have, what this slide calls ""high-risk medical condi...",NaN


In [127]:
# df['cluster'] = df.apply(lambda row: getCluster(df, row['cleaned']), axis=1)

# X_pred = X_pred))
def getCluster(row):
  return 0 if model(torch.from_numpy(np.stack(sbert_model.encode([row])[0]))) <= 0.5 else 1



In [126]:
df['cluster'] = 0

In [128]:
df['cluster'] = df.apply(lambda row: getCluster(row['full_text']), axis=1)

In [129]:
df.to_pickle('clusters.pickle')

In [135]:
df[df['cluster'] == 0]['location'].to_csv('clusters_0L.csv')

# Tokenization and modeling

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

## model 1: logistic regression



In [34]:
# before testing models, create a customed tokenizer. 
import spacy 

regexp = re.compile("(?u)\\b\\w\\w+\\b")
en_nlp=spacy.load('en')
old_tokenizer=en_nlp.tokenizer
en_nlp.tokenizer=lambda string: old_tokenizer.tokens_from_list (regexp.findall(string))

def custom_tokenizer(document):
    doc_spacy=en_nlp(document)
    return [token.lemma_ for token in doc_spacy]


In [ ]:
#split data into training (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
# run logistic regression on the whole labeled data:

pipe = make_pipeline(TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None),LogisticRegression())
param_grid={'logisticregression__C':[0.1,0.01,0.001],
           'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
           'tfidfvectorizer__min_df':[1,2,3,4,5]}
grid=GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is: ", grid.best_score_)
print("parameters of best cv score are: ", grid.best_params_)
print("the score on test set is: ", grid.score(X_test,y_test))

the best cv score is:  0.655515370705
parameters of best cv score are:  {'logisticregression__C': 0.01, 'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 1}
the score on test set is:  0.671480144404


In [ ]:
# run logistic regression on binary classifier data (negative and positive)

pipe = make_pipeline(TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None),LogisticRegression())
param_grid={'logisticregression__C':[0.1,0.01,0.001],
           'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
           'tfidfvectorizer__min_df':[1,2,3,4,5]}
grid=GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is: ", grid.best_score_)
print("parameters of best cv score are: ", grid.best_params_)
print("the score on test set is: ", grid.score(X_test_binary,y_test_binary))

the best cv score is:  0.813988095238
parameters of best cv score are:  {'logisticregression__C': 0.01, 'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__min_df': 1}
the score on test set is:  0.804733727811


In [ ]:
# try CountVectorizer instead of TfidfVectorizer and see which one is the best for featuring

vect=CountVectorizer(min_df=3, stop_words="english",tokenizer=custom_tokenizer,ngram_range=(1,1)).fit(X_train)
X_train=vect.transform(X_train)
param_grid={'C':[0.1,0.01,0.001]}
grid=GridSearchCV(LogisticRegression(), param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is: ",grid.best_score_)
X_test=vect.transform(X_test)
print("the score on test set is: ",grid.score(X_test,y_test))

the best cv score is:  0.641952983725
the score on test set is:  0.631768953069


- Based on the cv score and score on test set, CountVectorizer is not performing as well as TfidfVectorizer

## model 2: Support Vector Classifier (SVC)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
#SVC on 3-classifier data

pipe = make_pipeline(TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None),LinearSVC())
param_grid={'linearsvc__C':[1, 0.1,0.01,0.001],
           'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
           'tfidfvectorizer__min_df':[1,2,3,4,5]}
grid =GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)

print("parameters of best cv score are: ", grid.best_params_)
print("the score on test set is: ", grid.score(X_test,y_test))

the best cv score is:  0.662748643761
parameters of best cv score are:  {'tfidfvectorizer__ngram_range': (1, 1), 'linearsvc__C': 0.001, 'tfidfvectorizer__min_df': 1}
the score on test set is:  0.689530685921


In [37]:
#SVC on 2-classifier data
pipe = make_pipeline(TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None),LinearSVC())
param_grid={'linearsvc__C':[1, 0.1,0.01,0.001],
           'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
           'tfidfvectorizer__min_df':[1,2,3,4,5]}
grid =GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is: ", grid.best_score_)
print("parameters of best cv score are: ", grid.best_params_)
print("the score on test set is: ", grid.score(X_test_binary,y_test_binary))

TypeError: ignored

- Same here, in LinearSVC model, TfidfVectorizer performs better than CountVectorizer

## model 3: non-linear SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
# run non-linear SVC on 3-classifier data

vect = TfidfVectorizer(stop_words="english",ngram_range=(1,1),tokenizer=custom_tokenizer,norm=None)
param_grid={
            'svc__C':[0.01,0.1,1,10],
            'svc__gamma':[0.01,0.1,1,10],
            'tfidfvectorizer__min_df':[1,2,3]
}
pipe=make_pipeline(vect,SVC())
grid=GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test) )

the best cv score is : 0.644665461121
the best parameter is:  {'svc__C': 1, 'svc__gamma': 0.01, 'tfidfvectorizer__min_df': 3}
the score on the test set is:  0.635379061372


In [ ]:
# run non-linear SVC on 2-classifier data

vect = TfidfVectorizer(stop_words="english",ngram_range=(1,1),tokenizer=custom_tokenizer,norm=None)
param_grid={
            'svc__C':[0.01,0.1,1,10],
            'svc__gamma':[0.01,0.1,1,10],
            'tfidfvectorizer__min_df':[1,2,3]
}
pipe=make_pipeline(vect,SVC())
grid=GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary) )

the best cv score is : 0.782738095238
the best parameter is:  {'svc__C': 1, 'svc__gamma': 0.01, 'tfidfvectorizer__min_df': 3}
the score on the test set is:  0.751479289941


- SVM does not increase the accuracy score (linear models perform better with high dimensional data?)

## model 4: K nearest neighbors

In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [39]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
#run K nearest neighbor on 3-classifier data

vect =TfidfVectorizer(min_df=3, stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "kneighborsclassifier__n_neighbors":[1,5,10,15,20,30]
}
pipe=make_pipeline(vect,KNeighborsClassifier())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test) )

the best cv score is : 0.552441229656
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 4, 'kneighborsclassifier__n_neighbors': 1}
the score on the test set is:  0.595667870036


In [40]:
# run K nearest neighbor on 2-classifier data
vect =TfidfVectorizer(min_df=3, stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "kneighborsclassifier__n_neighbors":[1,5,10,15,20,30]
}
pipe=make_pipeline(vect,KNeighborsClassifier())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary) )

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in s

KeyboardInterrupt: ignored

- K nearest neighbors classifier by far has the lowest accuracy scores

## model 5: Nearest Centroid

In [ ]:
from sklearn.neighbors import NearestCentroid

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
#Run nearest centroid on 3-classifier data

vect =TfidfVectorizer( stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5]
}
pipe=make_pipeline(vect,NearestCentroid())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test))

the best cv score is : 0.661844484629
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 1}
the score on the test set is:  0.628158844765


In [ ]:
#Run nearest centroid on 2-classifier data

vect =TfidfVectorizer( stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5]
}
pipe=make_pipeline(vect,NearestCentroid())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary))

the best cv score is : 0.787202380952
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 2}
the score on the test set is:  0.804733727811


- the result of NearestCentroid is pretty good.

## Model 6: SGD + Linear SVM/Logistic Regression

In [ ]:
from sklearn.linear_model import SGDClassifier
# SGDClassifier is a linear classifiers (SVM, logistic regression, a.o.) with SGD training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
# run SGD + Linear SVM on 3-classifier data

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
sgd = SGDClassifier(fit_intercept=True,learning_rate='optimal')
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "sgdclassifier__alpha":[1,0.1,0.01,0.001]
}
pipe=make_pipeline(vect,sgd)
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test))

the best cv score is : 0.679023508137
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 1), 'sgdclassifier__alpha': 0.1, 'tfidfvectorizer__min_df': 2}
the score on the test set is:  0.620938628159


In [ ]:
# run SGD + Linear SVM on 2-classifier data

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
sgd = SGDClassifier(fit_intercept=True,learning_rate='optimal')
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "sgdclassifier__alpha":[1,0.1,0.01,0.001]
}
pipe=make_pipeline(vect,sgd)
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary))

the best cv score is : 0.821428571429
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'sgdclassifier__alpha': 1, 'tfidfvectorizer__min_df': 1}
the score on the test set is:  0.804733727811


In [ ]:
# run SGD + logistic regression on 3-classifier data

sgd_lr=SGDClassifier(loss= 'log',fit_intercept=True,learning_rate='optimal')
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "sgdclassifier__alpha":[1,0.1,0.01,0.001]
}
pipe=make_pipeline(vect,sgd_lr)
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test))

the best cv score is : 0.679023508137
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 1), 'sgdclassifier__alpha': 0.1, 'tfidfvectorizer__min_df': 2}
the score on the test set is:  0.631768953069


In [ ]:
# run SGD + logistic regression on 2-classifier data

sgd_lr=SGDClassifier(loss= 'log',fit_intercept=True,learning_rate='optimal')
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "sgdclassifier__alpha":[1,0.1,0.01,0.001]
}
pipe=make_pipeline(vect,sgd_lr)
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary))

the best cv score is : 0.825892857143
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 1), 'sgdclassifier__alpha': 0.1, 'tfidfvectorizer__min_df': 1}
the score on the test set is:  0.757396449704


- SGD + linear SVM or logistic regression provides slightly lower accuracy than without SGD

## Model 7: Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2)
X_train_binary,X_test_binary, y_train_binary,y_test_binary=train_test_split(data_binary["text"], data_binary["label"], test_size=0.2)

In [ ]:
# 3-classifier

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "multinomialnb__alpha":[0.1,1,10]
}
pipe=make_pipeline(vect,MultinomialNB())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)

print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test))

the best cv score is : 0.660940325497
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 1, 'multinomialnb__alpha': 10}
the score on the test set is:  0.660649819495


In [ ]:
#2-classifier

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "multinomialnb__alpha":[0.1,1,10]
}
pipe=make_pipeline(vect,MultinomialNB())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary))

the best cv score is : 0.809523809524
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__min_df': 1, 'multinomialnb__alpha': 10}
the score on the test set is:  0.786982248521


In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
#3-classifier 

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "bernoullinb__alpha":[0.1,1,10]
}
pipe=make_pipeline(vect,BernoulliNB())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train,y_train)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test,y_test))

the best cv score is : 0.649186256781
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 2), 'bernoullinb__alpha': 0.1, 'tfidfvectorizer__min_df': 1}
the score on the test set is:  0.649819494585


In [ ]:
#3-classifier 

vect =TfidfVectorizer(stop_words="english",tokenizer=custom_tokenizer,norm=None)
param_grid={
    "tfidfvectorizer__ngram_range":[(1,1),(1,2),(1,3)],
    'tfidfvectorizer__min_df':[1,2,3,4,5],
    "bernoullinb__alpha":[0.1,1,10]
}
pipe=make_pipeline(vect,BernoulliNB())
grid=GridSearchCV(pipe,param_grid,cv=StratifiedKFold(n_splits=10),scoring="accuracy")
grid.fit(X_train_binary,y_train_binary)
print("the best cv score is :", grid.best_score_)
print("the best parameter is: ", grid.best_params_)
print("the score on the test set is: ",grid.score(X_test_binary,y_test_binary))

the best cv score is : 0.799107142857
the best parameter is:  {'tfidfvectorizer__ngram_range': (1, 1), 'bernoullinb__alpha': 1, 'tfidfvectorizer__min_df': 2}
the score on the test set is:  0.798816568047


- naive bayes is performing well on the data

# Combining model results (not used in paper)

- The best results comes from logistic regression, linear SVC, multinomial naive bayes, and Bernolli naive bayes. Combine the result of these models and take the majority vote. 

In [ ]:
# use the parameters tuned before to predict training data by 4 models (if it's 2 vs 2, then it goes by the result from linear SVC)

# create a new dataframe
combined_train = pd.DataFrame(data["text"][:650])
combined_train["label"]=data["label"][:650]

combined_test = pd.DataFrame(data["text"][651:])
combined_test["label"]=data["label"][651:]

In [ ]:
combined_train.head()

,text,label
0,Are all u adults ready for your onslaught of vaccines? Say NO to Vaccine Mandate! NoMandates,-1.0
1,Kagro in the Morning: on Chapel Hill; vax roundup; King collapse; new AUMF state secrets,0.0
3,CANCER VACCINES??!!?!!?,0.0
4,Mexico's measles vaccination rate? 99%. The US'? 92%. Why immigrants aren't behind measles:,0.0
5,Vaccine stories not compelling without: aborted fetuses homosexuality autism Hitler fascism conspiracy promiscuity coverup U.N.,1.0


In [ ]:
combined_test.head()

,text,label
673,"Illinois announced EMERGENCY voting on measles vaccine for all, no exemptions The vote is tomorrow. you know, we... ht",0.0
674,Author of controversial vaccination/autism study shunned by Salem via,0.0
675,funnyordie: clippership: Finally! A safe option for anti-vaxxers. antivax,1.0
676,autism vax pharma Excellent. The audacity of pharma and MD's that say they know more about the mothers child...,-1.0
677,Africa reaches 6 months w/out a case of polio. We must continue to reach every child w/ the polio vaccine. endpolio,0.0


In [ ]:
# fill in the result of prediction by these models using the best parameters got above

#first, logistic regression

vect = TfidfVectorizer(ngram_range= (1,2), min_df=1, stop_words="english",tokenizer=custom_tokenizer,norm=None).fit(combined_train["text"])
train_vect = vect.transform(combined_train["text"])

logreg=LogisticRegression(C=0.01)
logreg.fit(train_vect,combined_train["label"])

test_vect=vect.transform(combined_test["text"])
logreg_result = logreg.predict(test_vect)
combined_test["logreg_result"]=logreg_result

In [ ]:
# second, linear SVC

vect = TfidfVectorizer(ngram_range= (1,1), min_df=1, stop_words="english",tokenizer=custom_tokenizer,norm=None).fit(combined_train["text"])
train_vect=vect.transform(combined_train["text"])

svc = LinearSVC(C=0.001)
svc.fit(train_vect,combined_train['label'])

test_vect=vect.transform(combined_test['text'])
svc_result=svc.predict(test_vect)
combined_test["svc_result"]=svc_result

In [ ]:
# third, multinomial naive bayes

vect = TfidfVectorizer(ngram_range= (1,2), min_df=1, stop_words="english",tokenizer=custom_tokenizer,norm=None).fit(combined_train["text"])
train_vect=vect.transform(combined_train["text"])

multinomialNB = MultinomialNB(alpha=10)
multinomialNB.fit(train_vect,combined_train['label'])

test_vect=vect.transform(combined_test['text'])
multinomialNB_result=multinomialNB.predict(test_vect)
combined_test["multinomialNB_result"]=multinomialNB_result

In [ ]:
# last, Bernoulli naive bayes

bernoulliNB=BernoulliNB(alpha=10)
bernoulliNB.fit(train_vect,combined_train['label'])

bernoulliNB_result=bernoulliNB.predict(test_vect)
combined_test["bernoulliNB_result"]=bernoulliNB_result

In [ ]:
combined_test.head(10)

,text,label,logreg_result,svc_result,multinomialNB_result,bernoulliNB_result
673,"Illinois announced EMERGENCY voting on measles vaccine for all, no exemptions The vote is tomorrow. you know, we... ht",0.0,1.0,1.0,1.0,0.0
674,Author of controversial vaccination/autism study shunned by Salem via,0.0,-1.0,-1.0,-1.0,0.0
675,funnyordie: clippership: Finally! A safe option for anti-vaxxers. antivax,1.0,1.0,1.0,1.0,0.0
676,autism vax pharma Excellent. The audacity of pharma and MD's that say they know more about the mothers child...,-1.0,-1.0,-1.0,-1.0,0.0
677,Africa reaches 6 months w/out a case of polio. We must continue to reach every child w/ the polio vaccine. endpolio,0.0,0.0,-1.0,1.0,0.0
678,The Anti-Vaccine Generation: How Movement Against Shots Got Its Start via,1.0,1.0,1.0,1.0,0.0
679,Powerful Anti-HIV Agent Can Work in a Vaccine,0.0,0.0,0.0,0.0,0.0
681,"No, only that I veered a bit from specific vaccine discussion, but you did that yourself too.",0.0,1.0,1.0,1.0,0.0
683,Ebola vaccine trial to start on volunteers in Liberia,0.0,0.0,0.0,0.0,0.0
684,How vaccinations may have opened up a new front in the GOP culture wars: (John Locher/AP),1.0,1.0,1.0,0.0,0.0


In [ ]:
result=combined_test.iloc[:,2:].mode(axis=1)

In [ ]:
result

,0,1
673,1.0,NaN
674,-1.0,NaN
675,1.0,NaN
676,-1.0,NaN
677,0.0,NaN
678,1.0,NaN
679,0.0,NaN
681,1.0,NaN
683,0.0,NaN
684,1.0,NaN


In [ ]:
n=0
for n in range(len(result)):
    if np.isnan(result.iloc[n,1]):
        result.set_value(n,0,combined_test.iloc[n,3])
    n = n + 1

In [ ]:
combined_test["majority_vote"]=result[0]